In [1]:
import os
import json
import openpyxl
from openpyxl.styles import Font

def flatten_json(y, parent_key='', sep='.'):
    """
    Função para achatar um JSON aninhado em um dicionário plano.
    """
    items = []
    if isinstance(y, list):
        # Se o JSON no nível superior for uma lista, processa cada item
        for i, item in enumerate(y):
            items.extend(flatten_json(item, f"{parent_key}[{i}]", sep=sep).items())
    elif isinstance(y, dict):
        # Se o JSON for um dicionário, processa normalmente
        for k, v in y.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, (dict, list)):
                # Processa recursivamente se for um dicionário ou lista
                items.extend(flatten_json(v, new_key, sep=sep).items())
            else:
                try:
                    # Tenta carregar o valor como JSON (caso seja um JSON em string)
                    parsed_value = json.loads(v) if isinstance(v, str) and v.strip().startswith(('{', '[')) else v
                    if isinstance(parsed_value, (dict, list)):
                        # Se o valor for um JSON válido, processa recursivamente
                        items.extend(flatten_json(parsed_value, new_key, sep=sep).items())
                    else:
                        items.append((new_key, parsed_value))
                except (ValueError, TypeError):
                    # Se não for um JSON válido, adiciona o valor diretamente
                    items.append((new_key, v))
    else:
        # Se o JSON for um valor simples (não lista ou dicionário)
        items.append((parent_key, y))
    return dict(items)

def map_json_to_excel(json_file, output_excel):
    """
    Função para mapear um arquivo JSON e gerar um Excel com o conteúdo mapeado.
    """
    try:
        # Carrega o arquivo JSON
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # Achata o JSON
        flattened_data = flatten_json(data)
        
        # Cria um novo workbook do Excel
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = "JSON Mapeado"
        
        # Adiciona cabeçalhos
        ws.append(["Chave", "Valor"] + [f"Parte {i+1}" for i in range(10)])  # Adiciona colunas extras para as partes da chave
        header_font = Font(bold=True)
        for cell in ws[1]:
            cell.font = header_font
        
        # Adiciona os dados mapeados
        for key, value in flattened_data.items():
            key_parts = key.split('.')  # Divide a chave pelo separador "."
            row = [key, value] + key_parts  # Adiciona as partes da chave como colunas extras
            ws.append(row)
        
        # Ajusta a largura das colunas
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter  # Obtém a letra da coluna
            for cell in column:
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except:
                    pass
            ws.column_dimensions[column_letter].width = max_length + 2
        
        # Salva o arquivo Excel
        wb.save(output_excel)
        print(f"Arquivo Excel gerado: {output_excel}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

# Caminho para o arquivo JSON e saída do Excel
json_file = r"c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report.json"
output_excel = r"c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\report_mapeado.xlsx"

# Executa o mapeamento
map_json_to_excel(json_file, output_excel)

Arquivo Excel gerado: c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\report_mapeado.xlsx


In [2]:
import pandas as pd
import json

def process_json(json_data):
    """
    Processa o JSON e gera as tabelas df_Sections, df_VisualContainers e df_VC_complemento.
    """
    # Listas para armazenar os dados das tabelas
    sections_data = []
    visual_containers_data = []
    vc_complemento_data = []

    # ID único para sections e visualContainers
    section_id_counter = 1
    visual_container_id_counter = 1

    # Processa as sections
    for section in json_data.get("sections", []):
        # Cria um ID único para a seção
        section_id = section_id_counter
        section_id_counter += 1

        # Adiciona os dados da seção à tabela df_Sections
        sections_data.append({
            "id_section": section_id,
            "sections": section.get("name"),
            "DisplayName": section.get("displayName"),
            "DisplayOption": section.get("displayOption"),
            "height": section.get("height"),
            "width": section.get("width"),
            "VisualContainers_id": [],
            "config": json.dumps(section)  # Inclui o JSON completo da seção no final
        })

        # Processa os visualContainers dentro da seção
        for visual_container in section.get("visualContainers", []):
            # Cria um ID único para o visualContainer
            visual_container_id = visual_container_id_counter
            visual_container_id_counter += 1

            # Adiciona o ID do visualContainer à seção correspondente
            sections_data[-1]["VisualContainers_id"].append(visual_container_id)

            # Processa o campo config do visualContainer
            config = json.loads(visual_container.get("config", "{}"))
            single_visual = config.get("singleVisual", {})
            prototype_query = single_visual.get("prototypeQuery", {}).get("Select", [])

            # Determina se é um grupo (visualType vazio)
            visual_type = single_visual.get("visualType", "")
            is_group = visual_type == ""

            # Adiciona os dados do visualContainer à tabela df_VisualContainers
            visual_containers_data.append({
                "VisualContainers_id": visual_container_id,
                "id_section": section_id,
                "height": visual_container.get("height"),
                "width": visual_container.get("width"),
                "x": visual_container.get("x"),
                "y": visual_container.get("y"),
                "z": visual_container.get("z"),
                "name": config.get("name"),
                "visualType": "group" if is_group else visual_type,
                "config": visual_container.get("config")  # Inclui o JSON completo no final
            })

            # Processa o prototypeQuery.Select para identificar colunas, medidas e agregações
            for select_item in prototype_query:
                tipo = None
                table = None
                column = None
                aggregation = None
                expression_type = None

                # Determina o tipo com base nos campos presentes
                if "Column" in select_item:
                    tipo = "column"
                    expression_type = "Column"
                    table = select_item["Column"].get("Expression", {}).get("SourceRef", {}).get("Entity")
                    column = select_item["Column"].get("Property")
                elif "Aggregation" in select_item:
                    tipo = "aggregation"
                    expression_type = "Aggregation"
                    aggregation_value = select_item["Aggregation"].get("Function")
                    aggregation_map = {1: "SUM", 2: "AVG", 3: "COUNT", 4: "MIN", 5: "MAX"}
                    aggregation = aggregation_map.get(aggregation_value, f"Unknown ({aggregation_value})")
                    table = select_item["Aggregation"].get("Expression", {}).get("SourceRef", {}).get("Entity")
                    column = select_item["Aggregation"].get("Expression", {}).get("Property")
                elif "Measure" in select_item:
                    tipo = "measure"
                    expression_type = "Measure"
                    table = select_item["Measure"].get("Expression", {}).get("SourceRef", {}).get("Entity")
                    column = select_item["Measure"].get("Property")

                # Adiciona os dados à tabela df_VC_complemento
                vc_complemento_data.append({
                    "VisualContainers_id": visual_container_id,
                    "tipo": tipo,
                    "Tabela": table,
                    "nome": column,
                    "nomeNoVisual": select_item.get("NativeReferenceName", column),
                    "Agregacao": aggregation,
                    "expression": expression_type
                })

    # Converte as listas em DataFrames
    df_sections = pd.DataFrame(sections_data)
    df_visual_containers = pd.DataFrame(visual_containers_data)
    df_vc_complemento = pd.DataFrame(vc_complemento_data)

    # Ajusta o campo VisualContainers_id em df_Sections para ser uma string (ou lista, se necessário)
    df_sections["VisualContainers_id"] = df_sections["VisualContainers_id"].apply(lambda x: ", ".join(map(str, x)))

    return df_sections, df_visual_containers, df_vc_complemento

# Caminho para o arquivo JSON
json_file = r"c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report.json"

# Carrega o JSON
with open(json_file, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Processa o JSON e gera as tabelas
df_sections, df_visual_containers, df_vc_complemento = process_json(json_data)

# Caminho para salvar o arquivo Excel
output_excel = r"c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\report_compilado.xlsx"

# Exporta os DataFrames para um único arquivo Excel
with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
    df_sections.to_excel(writer, sheet_name="Sections", index=False)
    df_visual_containers.to_excel(writer, sheet_name="VisualContainers", index=False)
    df_vc_complemento.to_excel(writer, sheet_name="VC_Complemento", index=False)

print(f"Arquivo Excel gerado: {output_excel}")

Arquivo Excel gerado: c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\report_compilado.xlsx


In [3]:
import pandas as pd
import json

def process_sections(json_data):
    """
    Processa a chave 'sections' e gera as tabelas df_sections, df_visual_containers e df_vc_complemento.
    """
    # Listas para armazenar os dados das tabelas
    sections_data = []
    visual_containers_data = []
    vc_complemento_data = []

    # ID único para sections e visualContainers
    section_id_counter = 1
    visual_container_id_counter = 1

    # Processa as sections
    for section in json_data.get("sections", []):
        # Cria um ID único para a seção
        section_id = section_id_counter
        section_id_counter += 1

        # Adiciona os dados da seção à tabela df_sections
        sections_data.append({
            "id_section": section_id,
            "sections": section.get("name"),
            "DisplayName": section.get("displayName"),
            "DisplayOption": section.get("displayOption"),
            "height": section.get("height"),
            "width": section.get("width"),
            "VisualContainers_id": [],
            "config": json.dumps(section)  # Inclui o JSON completo da seção no final
        })

        # Processa os visualContainers dentro da seção
        for visual_container in section.get("visualContainers", []):
            # Cria um ID único para o visualContainer
            visual_container_id = visual_container_id_counter
            visual_container_id_counter += 1

            # Adiciona o ID do visualContainer à seção correspondente
            sections_data[-1]["VisualContainers_id"].append(visual_container_id)

            # Processa o campo config do visualContainer
            config = json.loads(visual_container.get("config", "{}"))
            single_visual = config.get("singleVisual", {})
            prototype_query = single_visual.get("prototypeQuery", {}).get("Select", [])

            # Determina se é um grupo (visualType vazio)
            visual_type = single_visual.get("visualType", "")
            is_group = visual_type == ""

            # Adiciona os dados do visualContainer à tabela df_visual_containers
            visual_containers_data.append({
                "VisualContainers_id": visual_container_id,
                "id_section": section_id,
                "height": visual_container.get("height"),
                "width": visual_container.get("width"),
                "x": visual_container.get("x"),
                "y": visual_container.get("y"),
                "z": visual_container.get("z"),
                "name": config.get("name"),
                "visualType": "group" if is_group else visual_type,
                "config": visual_container.get("config")  # Inclui o JSON completo no final
            })

            # Processa o prototypeQuery.Select para identificar colunas, medidas e agregações
            for select_item in prototype_query:
                tipo = None
                table = None
                column = None
                aggregation = None
                expression_type = None

                # Determina o tipo com base nos campos presentes
                if "Column" in select_item:
                    tipo = "column"
                    expression_type = "Column"
                    table = select_item["Column"].get("Expression", {}).get("SourceRef", {}).get("Entity")
                    column = select_item["Column"].get("Property")
                elif "Aggregation" in select_item:
                    tipo = "aggregation"
                    expression_type = "Aggregation"
                    aggregation_value = select_item["Aggregation"].get("Function")
                    aggregation_map = {1: "SUM", 2: "AVG", 3: "COUNT", 4: "MIN", 5: "MAX"}
                    aggregation = aggregation_map.get(aggregation_value, f"Unknown ({aggregation_value})")
                    table = select_item["Aggregation"].get("Expression", {}).get("SourceRef", {}).get("Entity")
                    column = select_item["Aggregation"].get("Expression", {}).get("Property")
                elif "Measure" in select_item:
                    tipo = "measure"
                    expression_type = "Measure"
                    table = select_item["Measure"].get("Expression", {}).get("SourceRef", {}).get("Entity")
                    column = select_item["Measure"].get("Property")

                # Adiciona os dados à tabela df_vc_complemento
                vc_complemento_data.append({
                    "VisualContainers_id": visual_container_id,
                    "tipo": tipo,
                    "Tabela": table,
                    "nome": column,
                    "nomeNoVisual": select_item.get("NativeReferenceName", column),
                    "Agregacao": aggregation,
                    "expression": expression_type
                })

    # Converte as listas em DataFrames
    df_sections = pd.DataFrame(sections_data)
    df_visual_containers = pd.DataFrame(visual_containers_data)
    df_vc_complemento = pd.DataFrame(vc_complemento_data)

    return df_sections, df_visual_containers, df_vc_complemento

# Caminho para o arquivo JSON


In [4]:
def process_config(json_data, df_sections, df_visual_containers):
    """
    Processa a chave 'config' e gera as tabelas df_bookmarks e df_bookmark_detalhamento,
    incluindo bookmarks com children, mantendo a hierarquia correta.
    """
    # Listas para armazenar os dados das tabelas
    bookmarks_data = []
    bookmark_detalhamento_data = []

    # IDs únicos para bookmarks
    bookmark_id_counter = 1

    # Processa os bookmarks na chave 'config'
    config = json.loads(json_data.get("config", "{}"))
    bookmarks = config.get("bookmarks", [])

    def process_bookmark(bookmark, parent_id=None, parent_name=None, child_index=None):
        nonlocal bookmark_id_counter

        # Define o nome do pai e o ID do pai
        id_pai = parent_id
        nome_pai = parent_name

        current_id = bookmark_id_counter  # Salva o ID atual do bookmark
        bookmark_id_counter += 1  # Incrementa o contador de IDs  
        
        # Verifica se o campo options existe e extrai os subcampos
        options = bookmark.get("options", {})
        apply_only_to_target_visuals = options.get("applyOnlytoTargetVisuals", False)
        supress_data = options.get("suppressData", False)

        # Extrai a seção do explorationState
        exploration_state = bookmark.get("explorationState", {})
        sections_keys = list(exploration_state.get("sections", {}).keys())
        section_name = sections_keys[0] if sections_keys else None  # Verifica se há seções antes de acessar

        # Relaciona os visualContainers ao id_bookmark
        if section_name:  # Apenas processa se houver uma seção
            visual_containers_data = exploration_state.get("sections", {}).get(section_name, {})
            visual_containers = visual_containers_data.get("visualContainers", {})
            visual_containers_groups = visual_containers_data.get("visualContainerGroups", {})

            # Processa VisualContainers
            for visual_container_name in visual_containers:
                # Busca o section_id correspondente na tabela df_sections
                section_row = df_sections[df_sections["sections"] == section_name]
                section_id = section_row.iloc[0]["id_section"] if not section_row.empty else None

                # Busca o visualContainers_id correspondente na tabela df_visual_containers
                visual_container_row = df_visual_containers[df_visual_containers["name"] == visual_container_name]
                visual_container_id = visual_container_row.iloc[0]["VisualContainers_id"] if not visual_container_row.empty else None

                # Adiciona o detalhamento ao df_bookmark_detalhamento
                bookmark_detalhamento_data.append({
                    "id_bookmark": current_id,
                    "section_name": section_name,
                    "visual_container_name": visual_container_name,
                    "section_id": section_id,
                    "visualContainers_id": visual_container_id,
                    "type": "VisualContainer"
                })

            # Processa visualContainersGroup
            for visual_container_group_name in visual_containers_groups:
                # Busca o section_id correspondente na tabela df_sections
                section_row = df_sections[df_sections["sections"] == section_name]
                section_id = section_row.iloc[0]["id_section"] if not section_row.empty else None

                # Busca o visualContainers_id correspondente na tabela df_visual_containers
                visual_container_row = df_visual_containers[df_visual_containers["name"] == visual_container_group_name]
                visual_container_id = visual_container_row.iloc[0]["VisualContainers_id"] if not visual_container_row.empty else None

                # Adiciona o detalhamento ao df_bookmark_detalhamento
                bookmark_detalhamento_data.append({
                    "id_bookmark": current_id,
                    "section_name": section_name,
                    "visual_container_name": visual_container_group_name,
                    "section_id": section_id,
                    "visualContainers_id": visual_container_id,
                    "type": "VisualContainerGroup"
                })

        # Processa os children, se existirem
        children = bookmark.get("children", [])
        for index, child in enumerate(children):
            process_bookmark(
                child,
                parent_id=current_id,
                parent_name=bookmark.get("name"),
                child_index=index
            )

    # Processa cada bookmark
    for index, bookmark in enumerate(bookmarks):
        process_bookmark(bookmark, child_index=index)

    # Converte as listas em DataFrames
    df_bookmarks = pd.DataFrame(bookmarks_data)
    df_bookmark_detalhamento = pd.DataFrame(bookmark_detalhamento_data)

    return df_bookmarks, df_bookmark_detalhamento

In [5]:
def process_config(json_data, df_sections, df_visual_containers):
    """
    Processa a chave 'config' e gera as tabelas df_bookmarks e df_bookmark_detalhamento,
    incluindo bookmarks com children, mantendo a hierarquia correta.
    """
    # Listas para armazenar os dados das tabelas
    bookmarks_data = []
    bookmark_detalhamento_data = []

    # IDs únicos para bookmarks
    bookmark_id_counter = 1

    # Processa os bookmarks na chave 'config'
    config = json.loads(json_data.get("config", "{}"))

    bookmarks = config.get("bookmarks", [])

    def process_bookmark(bookmark, parent_id=None, parent_name=None, child_index=None):
        nonlocal bookmark_id_counter

        # Define o nome do pai e o ID do pai
        id_pai = parent_id
        nome_pai = parent_name

        current_id = bookmark_id_counter  # Salva o ID atual do bookmark
        bookmark_id_counter += 1  # Incrementa o contador de IDs  


        # Verifica se o campo options existe e extrai os subcampos
        options = bookmark.get("options", {})
        apply_only_to_target_visuals = options.get("applyOnlytoTargetVisuals", False)
        supress_data = options.get("suppressData", False)

        # Adiciona os dados do bookmark à lista
        bookmarks_data.append({
            "id_bookmark": current_id,
            "name": bookmark.get("name"),
            "displayName": bookmark.get("displayName"),
            "parent_id": id_pai,
            "parent_name": nome_pai,
            "child_index": child_index,
            "applyOnlyToTargetVisuals": apply_only_to_target_visuals,
            "suppressData": supress_data
        })

        # Processa os children, se existirem
        children = bookmark.get("children", [])
        for index, child in enumerate(children):
            process_bookmark(
                child,
                parent_id=current_id,
                parent_name=bookmark.get("name"),
                child_index=index
            )

    # Processa cada bookmark
    for index, bookmark in enumerate(bookmarks):
        process_bookmark(bookmark, child_index=index)

    # Converte as listas em DataFrames
    df_bookmarks = pd.DataFrame(bookmarks_data)
    df_bookmark_detalhamento = pd.DataFrame(bookmark_detalhamento_data)

    return df_bookmarks, df_bookmark_detalhamento

df_bookmarks, df_bookmark_detalhamento = process_config(json_data,df_sections,df_visual_containers)

In [6]:

def explore_visual_containers(data):
    """
    Explora o JSON no caminho config -> bookmarks -> explorationState -> sections -> visualContainers
    e converte os dados em uma tabela estruturada (DataFrame).
    """
    rows = []

    # Verifica se o campo config existe e é um JSON válido
    config = json.loads(data.get("config", "{}"))
    bookmarks = config.get("bookmarks", [])

    # Itera sobre os bookmarks
    for bookmark in bookmarks:
        bookmark_name = bookmark.get("displayName", "Sem Nome")
        bookmark_id = bookmark.get("name", "Sem ID")
        exploration_state = bookmark.get("explorationState", {})
        sections = exploration_state.get("sections", {})

        # Itera sobre as sections dentro do explorationState
        for section_id, section_data in sections.items():
            visual_containers = section_data.get("visualContainers", {})

            # Itera sobre os visualContainers
            for vc_id, vc_data in visual_containers.items():
                filters = vc_data.get("filters", {}).get("byExpr", [])
                single_visual = vc_data.get("singleVisual", {})
                visual_type = single_visual.get("visualType", "Desconhecido")

                # Itera sobre os filtros (byExpr)
                for filter_item in filters:
                    filter_type = filter_item.get("type", "Desconhecido")
                    expression = filter_item.get("expression", {})
                    column_data = expression.get("Column", {}) or \
                        expression.get("Aggregation", {}).get("Expression", {}).get("Column", {}) or \
                        expression.get("Measure", {})
                    table = column_data.get("Expression", {}).get("SourceRef", {}).get("Entity", "Desconhecido") 
                    property_name = column_data.get("Property", "Desconhecido")

                    # Adiciona os dados à lista
                    rows.append({
                        "Bookmark Name": bookmark_name,
                        "Bookmark ID": bookmark_id,
                        "Section ID": section_id,
                        "VisualContainer ID": vc_id,
                        "Visual Type": visual_type,
                        "Filter Type": filter_type,
                        "Table": table,
                        "Column": property_name
                    })

    # Converte os dados em um DataFrame
    return pd.DataFrame(rows)

# Caminho para o arquivo JSON
json_file = r"c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report.json"

# Carrega o JSON
with open(json_file, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Explora os visualContainers e cria a tabela
df_visual_containers = explore_visual_containers(json_data)

In [7]:
"""processed_rows = []

# Itera sobre os dados do DataFrame df_visual_byexpr
for index, row in df_visual_byexpr.iterrows():
    by_expr_item = json.loads(row['byExprItem'])  # Converte o JSON string de volta para dicionário
    item_type = by_expr_item.get('type')  # Obtém o campo 'type'
    expression = by_expr_item.get('expression', {})  # Obtém o campo 'expression'
    source_ref = expression.get('Column', {}).get('Expression', {}).get('SourceRef', {}).get('Entity')  # Obtém o 'sourceRef'
    property_name = expression.get('Column', {}).get('Property')  # Obtém o 'property'
    
    # Adiciona os dados processados à lista
    processed_rows.append({
        'visualName': row['visualName'],
        'visualType': row['visualType'],
        'type': item_type,
        'sourceRef': source_ref,
        'property': property_name,
        'byExprIndex': index  # Índice do item da byExpr
    })

# Converte a lista em um novo DataFrame
df_processed_byexpr = pd.DataFrame(processed_rows)

# Exibe o DataFrame resultante
df_processed_byexpr"""


"processed_rows = []\n\n# Itera sobre os dados do DataFrame df_visual_byexpr\nfor index, row in df_visual_byexpr.iterrows():\n    by_expr_item = json.loads(row['byExprItem'])  # Converte o JSON string de volta para dicionário\n    item_type = by_expr_item.get('type')  # Obtém o campo 'type'\n    expression = by_expr_item.get('expression', {})  # Obtém o campo 'expression'\n    source_ref = expression.get('Column', {}).get('Expression', {}).get('SourceRef', {}).get('Entity')  # Obtém o 'sourceRef'\n    property_name = expression.get('Column', {}).get('Property')  # Obtém o 'property'\n    \n    # Adiciona os dados processados à lista\n    processed_rows.append({\n        'visualName': row['visualName'],\n        'visualType': row['visualType'],\n        'type': item_type,\n        'sourceRef': source_ref,\n        'property': property_name,\n        'byExprIndex': index  # Índice do item da byExpr\n    })\n\n# Converte a lista em um novo DataFrame\ndf_processed_byexpr = pd.DataFrame(pro

In [8]:
json_file = r"c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report.json"

# Carrega o JSON
with open(json_file, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Processa as sections e config
df_sections, df_visual_containers, df_vc_complemento = process_sections(json_data)
df_bookmarks, df_bookmark_detalhamento = process_config(json_data,df_sections,df_visual_containers)

df_visual_containers_complemento = explore_visual_containers(json_data)
# Caminho para salvar o arquivo Excel
output_excel = r"c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\report_compilado.xlsx"

# Exporta os DataFrames para um único arquivo Excel
with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
    df_sections.to_excel(writer, sheet_name="Sections", index=False)
    df_visual_containers.to_excel(writer, sheet_name="VisualContainers", index=False)
    df_visual_containers_complemento.to_excel(writer, sheet_name="VisualContainers_complemento", index=False)
    df_vc_complemento.to_excel(writer, sheet_name="VC_Complemento", index=False)
    df_bookmarks.to_excel(writer, sheet_name="Bookmarks", index=False)
    df_bookmark_detalhamento.to_excel(writer, sheet_name="Bookmark_Detalhamento", index=False)

print(f"Arquivo Excel gerado: {output_excel}")

Arquivo Excel gerado: c:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\report_compilado.xlsx


In [9]:
import json

# Caminho para o arquivo JSON original
input_file = r'C:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report.json'
output_file = r'C:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report_modified.json'

# Carregar o arquivo JSON original
with open(input_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Verificar se a chave "config" existe e é uma string
if "config" in data and isinstance(data["config"], str):
    # Desserializar o valor da chave "config" (JSON dentro de uma string)
    config = json.loads(data["config"])
    
    # Localizar a lista de bookmarks
    bookmarks = config.get("bookmarks", [])

    # Verificar se a lista de bookmarks existe
    if isinstance(bookmarks, list):
        # Procurar o bookmark com displayName 'P2_PT_Bar' e id '77061bd1b5ebe593a6b1'
        bookmark_found = False
        for bookmark in bookmarks:
            if bookmark.get('displayName') == 'P2_PT_Bar' and bookmark.get('name') == '77061bd1b5ebe593a6b1':
                # Fazer uma cópia do bookmark
                new_bookmark = bookmark.copy()
                # Alterar os valores solicitados
                new_bookmark['displayName'] = 'SUPER_TESTE'
                new_bookmark['name'] = ''
                # Adicionar o novo bookmark ao final da lista
                bookmarks.append(new_bookmark)
                bookmark_found = True
                break

        if not bookmark_found:
            print("Bookmark com displayName 'P2_PT_Bar' e id '77061bd1b5ebe593a6b1' não encontrado.")
    else:
        print("A chave 'bookmarks' não é uma lista ou não existe no JSON dentro de 'config'.")

    # Serializar novamente o dicionário "config" para uma string JSON
    data["config"] = json.dumps(config)
else:
    print("A chave 'config' não existe ou não é uma string no arquivo JSON.")

# Salvar o arquivo modificado em uma nova cópia
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

print(f"Arquivo modificado salvo como '{output_file}'")

Arquivo modificado salvo como 'C:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report_modified.json'
